In [ ]:
import os
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

Dataset Prepartion is from the Kernel: https://www.kaggle.com/rohanrao/tutorial-on-reading-large-datasets

In [ ]:
%%time
train = pd.read_pickle('../input/riiid-trainpkl/riiid_train.pkl.gzip')
print(train.shape)
train.head()

In [ ]:
train.info()

In [ ]:
train.isna().sum()

In [ ]:
print("Number Of Users", len(train.user_id.unique()))

In [ ]:
train.content_type_id.value_counts()

In [ ]:
ax=sns.countplot(train.content_type_id)
ax.set(xlabel='Question/Lecture Content', ylabel='Count')

Number of Questions posed releatively high compared to the lectures.

Total Posed Question - 99271300

Total Watched lectures - 1959032

# Question is Answered Correctly or not

This is the Target we want to find the prediction.

answered_correctly(0) --> wrong Answer

answered_correctly(1) --> Right Answer

In [ ]:
train.answered_correctly.value_counts()

In [ ]:
sns.countplot(train.answered_correctly)

In [ ]:
print('Percent Of Questions Answered Correctly:', 
      100*(len(train[train.answered_correctly==1])/len(train[train.answered_correctly>=0]))
     )

Arount 65.7% of the Questions are Answered Correctly in the dataset.

# user_answer

In [ ]:
train.user_answer.value_counts()

In [ ]:
sns.countplot(train.user_answer)

In [ ]:
plt.pie(
    list(train.user_answer.value_counts()),
    labels=list(train.user_answer.value_counts().index),
    shadow=True,
    autopct='%1.1f%%'
)


In [ ]:
100*train[train.user_answer>=0].user_answer.value_counts()/len(train[train.user_answer>=0])


Multi choice questions

* Percentage of the Answer choices the User hits for option:2 is reletively less compared to the other options

# TimeStamp

In [ ]:
milliseconds_to_days=1000*60*60*24

In [ ]:
train.head()

In [ ]:
print(max(train.timestamp/milliseconds_to_days))

Max Number Of days user is active after first interaction : 1011 ~2.8 Years

In [ ]:
%%time
(train[['timestamp']]/milliseconds_to_days).hist(bins=100)

 The above Histogram represents the Number Of Interactions as the days preceed.
* 
* We can observe a sharp fall in the histogram starting around ~80 days
* Looks like only a the number of interactions got reduced

In [ ]:
train.user_id.unique()[:20]

In [ ]:
plt.plot(train[train.user_id == 115].timestamp/milliseconds_to_days)

In [ ]:
print('Number Of Samples For user:115', train[train.user_id == 115].shape)


Looks the user interaction of 2 days is stored.

lets look at other user.

In [ ]:
print(train[train.user_id == 124].shape)

In [ ]:
plt.plot(train[train.user_id == 124].timestamp)

In [ ]:
print(train[train.user_id == 8623].shape)
plt.plot(train[train.user_id == 8623].timestamp/milliseconds_to_days)

In [ ]:
print(train[train.user_id == 81429].shape)
plt.plot(train[train.user_id == 81429].timestamp/milliseconds_to_days)

Number Of Records for the User: 8623 --> 112

Days Gap Between first and last visit: 10 days

We can see some users are more consistent in days than the other.

1. timestamp indicates if the user is new/old.
2. User Interacting frequently/not
3. Paired with the Questions answered; Interacting single day for a long time or not etc.

# lets Check the content_id

In [ ]:
train.groupby('content_type_id')[['content_id']].nunique()

questions - 13523
lectures - 415

1. Any Questions/Lectures Re-attempted?
2. Top 20 questions answered correct/wrong
3. Is there a tough question that many users atteps answered wrong.


In [ ]:

content_id_df=train[train.content_type_id==0].groupby('content_id')[['user_id', 'answered_correctly']].agg({'user_id': 'nunique', 'answered_correctly': 'sum'}).reset_index()
content_id_df.rename(columns={'user_id': 'total_answered'}, inplace=True)
content_id_df['percent_answered_correctly']=100*content_id_df['answered_correctly'].div(content_id_df['total_answered'])
content_id_df.head()


In [ ]:
content_id_df.head()

In [ ]:
content_id_df.corr()

In [ ]:
sns.heatmap(content_id_df.corr())

In [ ]:
sns.scatterplot(data=content_id_df,
                x='total_answered',
                y='answered_correctly',
                hue=[0]*len(content_id_df),
                palette='CMRmap_r'
               )

In [ ]:
sns.scatterplot(data=content_id_df,
                x='total_answered',
                y='percent_answered_correctly',
                hue=[0]*len(content_id_df),
                palette='CMRmap_r'
               )

In [ ]:
content_id_df[content_id_df.total_answered>=75000].shape

In [ ]:
content_id_df[content_id_df.total_answered<=10].shape

In [ ]:
content_id_df[content_id_df.total_answered<=100].shape

In [ ]:
sns.distplot(content_id_df[(content_id_df.total_answered<=50000) 
                          ].total_answered,
             bins=200,
             kde=False)

1. Some of the questions are answered more than one time by the user --> with percent_answered_correctly>100

2. There is a high amount of variance for the question answered less.

3. 44 Questions answered by more than 75000 users.

4. Observed that the variance reduction in the graph as move from 0-->50000 on y-axis

In [ ]:
train[train.content_type_id==0].groupby(['user_id', ''])[['content_id']].count()

In [ ]:
sns.pairplot(train[['']])